In [243]:
import pandas as pd
import numpy as np
import matplotlib as plt
import tqdm
from datetime import datetime, date, time
import seaborn as sns
import os

Для начала загрузим csv файл с признаками

In [244]:
with open('/content/drive/MyDrive/bd_git/features.csv') as f:
    df_features = pd.read_csv(f)

In [245]:
df_features[['namespaceName', 'commitsСount', 'activeDaysCount', 'activeWeeksCount', 'activeMonthCount', 'additionsSum', 'deletionsSum']]

,namespaceName,commitsСount,activeDaysCount,activeWeeksCount,activeMonthCount,additionsSum,deletionsSum
0,1037 Создание пользовательского интерфейса для...,2,2,2,1,10,4
1,806 Клиентская часть системы управления микрок...,32,11,8,6,5329,383
2,511 Разработка программной части системы управ...,3,2,2,2,207,3
3,19102 Телецентр МИЭМ,48,10,7,4,814,2607
4,246 Телевидение МИЭМ,22,3,3,2,2284,1346
...,...,...,...,...,...,...,...
415,811 Многофункциональный сценарный контроллер о...,4,3,2,1,371,53
416,797 Масштабируемая система видеоконференций дл...,1,1,1,1,7,5
417,1081 Разработка мультимодального человеко маши...,2,1,1,1,1185,0
418,1010 Исследование возможности восстановления с...,3,1,1,1,2,0


In [246]:
df_features = df_features.drop(columns=['Unnamed: 0'])

Изменим столбец с названием проекта, оставив в нем только его номер, а также переименуем некоторые стобцы для того что в будущем соединить две таблицы по ним

In [247]:
df_features['namespaceName'] = df_features['namespaceName'].apply(lambda x: x.split()[0])
df_features = df_features.rename(columns={"name": "student", "namespaceName": "projnum"})

Изменим тип колонки с номером проекта

In [248]:
df_features = df_features.astype({'projnum': 'int32'})

Загрузим датафрейм с оценками студентов за проекты, перед этим вручную отформатировав excel таблички, которые заполнялись тоже руками, вследствии чего дальнейшая работа с ними была бы невозможной без предобработки данных и их исправления

In [256]:
df_marks = pd.DataFrame(columns=['projnum', 'student', 'mark'])

In [257]:
for table in os.listdir('/content/drive/MyDrive/bd_git/marks/'):
  with open('/content/drive/MyDrive/bd_git/marks/' + table) as f:
     df_marks = pd.concat([df_marks, pd.read_csv(f, sep=';')[['projnum', 'student', 'mark']]])

Посмотрим на целевой признак mark

In [258]:
df_marks[['projnum', 'mark']].describe(include='all')

,projnum,mark
count,891,886.0
unique,354,11.0
top,246,8.0
freq,12,267.0


Мы имеем 5 пустых значений. Их немного, поэтому мы можем смело их удалить

In [260]:
df_marks = df_marks[df_marks['mark'].notna()]
df_marks[['projnum', 'mark']].describe(include='all')

,projnum,mark
count,886,886.0
unique,352,11.0
top,246,8.0
freq,12,267.0


Изменим тип целевого признака

In [261]:
df_marks = df_marks.astype({'mark': 'int32'})

Соединим две таблицы по двум значениям - имени студента и номеру его проекта

In [263]:
df = df_features.merge(df_marks,on=['student','projnum'], how='inner')

In [265]:
df[['projnum', 'commitsСount', 'activeDaysCount', 'activeWeeksCount', 'activeMonthCount', 'additionsSum', 'deletionsSum','mark']]

,projnum,commitsСount,activeDaysCount,activeWeeksCount,activeMonthCount,additionsSum,deletionsSum,mark
0,806,32,11,8,6,5329,383,4
1,19109,91,23,14,5,31363,46883,8
2,318,5,4,3,3,186,34,8
3,964,8,1,1,1,850206,0,8
4,608,19,5,4,3,68927,6764,9
...,...,...,...,...,...,...,...,...
212,1058,7,1,1,1,1300,563,7
213,999,1,1,1,1,3,0,7
214,539,5,2,1,1,157,0,4
215,1081,2,1,1,1,1185,0,8
